In [1]:
import skimage
import os
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib
import matplotlib.pyplot as plt
from skimage import data
from skimage import io
import glob
from skimage.viewer import ImageViewer
import cv2
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
%matplotlib inline

C:\Users\karan\Documents\Softwares\lib\site-packages\skimage\viewer\utils\core.py:10: UserWarning: Recommended matplotlib backend is `Agg` for full skimage.viewer functionality.
  warn("Recommended matplotlib backend is `Agg` for full "


In [2]:
#Define List of Users and define File-Path to Users Folder
list_users = ['athenaprocopiou', 'daftcollectionofficial', 'dodobaror', 'emporiosirenuse', 'heidikleinswim', 'lisamariefernandez', 
          'loupcharmant', 'miguelinagambaccini', 'muzungusisters', 'zeusndione', 'zimmermann']

path = "C:/Users/karan/Desktop/Dropbox/Harvard/Capstone_EmpSirenuse/CompanyScrape/"
dest_path = r"C:/Users/karan/Desktop/Dropbox/Harvard/Capstone_EmpSirenuse/Clustering_"

In [3]:
#Helper Function to get images from path
def get_images(path, list_users):
    #Get all Images of Users in the List
    user_imgs = {}
    for i in list_users:
        temp_path = path + i
        for j in glob.glob(temp_path + '/*.jpg'):
            file_name = j.replace(temp_path,'')[1:]
            img = io.imread(j)
            user_imgs[(i,file_name)] = img
    return user_imgs 


#Get Images from User List and Path
user_imgs = get_images(path, list_users)
print("Number of images loaded:", len(user_imgs))

Number of images loaded: 18372


In [4]:
#Helper function to convert image to d-dimension vector
def convert(user_imgs):
    user_features = []
    for i in user_imgs.items():
        r_mean, r_std, r_med = np.mean(i[1][:,:,0].ravel()), np.std(i[1][:,:,0].ravel()), np.median(i[1][:,:,0].ravel())
        g_mean, g_std, g_med  = np.mean(i[1][:,:,1].ravel()), np.std(i[1][:,:,1].ravel()), np.median(i[1][:,:,1].ravel())
        b_mean, b_std, b_med  = np.mean(i[1][:,:,2].ravel()), np.std(i[1][:,:,2].ravel()), np.median(i[1][:,:,2].ravel())
        canny = np.mean(np.ravel(cv2.Canny(cv2.cvtColor(i[1], cv2.COLOR_BGR2HSV),100,200,L2gradient = True)))
        user_features.append(np.array([i[0][0],i[0][1], r_mean, r_std, r_med, g_mean, g_std, g_med, b_mean, b_std, b_med, canny]))
    return user_features
    
#Convert Images
user_features = convert(user_imgs)
print("Feature Vectors Created.")

Feature Vectors Created.


In [5]:
#Create Dataframe of Features
df = pd.DataFrame(user_features)
df.columns = ["User_Handle","URL","R_Mean", "R_STD", "R_MED", "G_Mean", "G_STD", "G_MED", "B_Mean", "B_STD", "B_MED", "Canny"]
display(df.head())

,User_Handle,URL,R_Mean,R_STD,R_MED,G_Mean,G_STD,G_MED,B_Mean,B_STD,B_MED,Canny
0,athenaprocopiou,10004086_104906123181355_768209000_n.jpg,107.210754395,56.560502417,114.0,122.385576172,62.206188484,137.0,148.272832031,69.4661467418,166.0,43.5896850586
1,athenaprocopiou,10004323_527242954060014_180015438_n.jpg,102.749882813,54.3744882338,90.0,133.589592285,37.4285485383,132.0,173.389179687,38.1952559522,183.0,11.2253540039
2,athenaprocopiou,10005195_1478590019019559_1318206042_n.jpg,178.573417969,60.2873264372,197.0,168.81440918,56.4889614268,178.0,150.826479492,51.1980452928,160.0,15.7669189453
3,athenaprocopiou,10005738_1001124636606760_1174146246_n.jpg,181.720700446,63.6578826258,212.0,203.222022462,46.7607726791,221.0,205.265769033,43.9047388404,221.0,24.487345679
4,athenaprocopiou,10009138_1440188279552462_1043878933_n.jpg,178.203747559,43.5411449307,186.0,179.226801758,47.8567300161,192.0,168.808415527,38.9282998132,178.0,23.2955200195


In [6]:
#Define Cluster Count
cluster_count = 7

#Define Dataset
data = df.copy(deep=True)
del data["User_Handle"]
del data["URL"]

#Implement K-Means Algortihm
model_kmeans = KMeans(n_clusters=cluster_count, random_state=9001)

#Fit Model and Predict
model_kmeans.fit(data)
y_pred = model_kmeans.predict(data)
df['Prediction'] = model_kmeans.labels_
display(df.head())

,User_Handle,URL,R_Mean,R_STD,R_MED,G_Mean,G_STD,G_MED,B_Mean,B_STD,B_MED,Canny,Prediction
0,athenaprocopiou,10004086_104906123181355_768209000_n.jpg,107.210754395,56.560502417,114.0,122.385576172,62.206188484,137.0,148.272832031,69.4661467418,166.0,43.5896850586,4
1,athenaprocopiou,10004323_527242954060014_180015438_n.jpg,102.749882813,54.3744882338,90.0,133.589592285,37.4285485383,132.0,173.389179687,38.1952559522,183.0,11.2253540039,4
2,athenaprocopiou,10005195_1478590019019559_1318206042_n.jpg,178.573417969,60.2873264372,197.0,168.81440918,56.4889614268,178.0,150.826479492,51.1980452928,160.0,15.7669189453,5
3,athenaprocopiou,10005738_1001124636606760_1174146246_n.jpg,181.720700446,63.6578826258,212.0,203.222022462,46.7607726791,221.0,205.265769033,43.9047388404,221.0,24.487345679,1
4,athenaprocopiou,10009138_1440188279552462_1043878933_n.jpg,178.203747559,43.5411449307,186.0,179.226801758,47.8567300161,192.0,168.808415527,38.9282998132,178.0,23.2955200195,1


In [7]:
#Helper function to create folders for Image Clustering
def save_clusters(df, user_imgs, dest_path, label):
    dest_path += label + "/"
    for i in user_imgs.items():
        temp_row = df[df["URL"]==i[0][1]]
        name, pred_folder = i[0][1], str(temp_row['Prediction'].values[0])
        temp_path = dest_path + "Cluster" + pred_folder + "/"
        if not os.path.exists(temp_path):
            os.makedirs(temp_path)
        io.imsave(temp_path+name, i[1])

save_clusters(df, user_imgs, dest_path, "Competitors")
print("All Images Clustered.")

C:\Users\karan\Documents\Softwares\lib\site-packages\skimage\io\_io.py:132: UserWarning: C:/Users/karan/Desktop/Dropbox/Harvard/Capstone_EmpSirenuse/Clustering_Competitors/Cluster2/21688773_1968249120130958_7257106618284769280_n.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
C:\Users\karan\Documents\Softwares\lib\site-packages\skimage\io\_io.py:132: UserWarning: C:/Users/karan/Desktop/Dropbox/Harvard/Capstone_EmpSirenuse/Clustering_Competitors/Cluster2/26866626_176586996287906_7065361033170255872_n.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
C:\Users\karan\Documents\Softwares\lib\site-packages\skimage\io\_io.py:132: UserWarning: C:/Users/karan/Desktop/Dropbox/Harvard/Capstone_EmpSirenuse/Clustering_Competitors/Cluster2/10891000_575949152542012_1261301453_n.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
C:\Users\karan\Documents\Softwares\lib\site-packages\skimage\io\_io.py:132: UserWarning: C:/Users/karan/D

All Images Clustered.


In [19]:
#Percentage of Cluster Presence
cluster_presence = []
for i in list(df['User_Handle'].unique()):
    user_dict = {}
    temp_df = df[df['User_Handle']==i]
    post_count  =  len(temp_df)
    user_dict['User'] = i
    for j in sorted(list(temp_df['Prediction'].unique())):
        user_dict["Cluster_"+str(j)] = len(temp_df[temp_df['Prediction']==j])*100/post_count
    cluster_presence.append(user_dict)

#Create Cluster Presence Dataframe
df_presence = pd.DataFrame(cluster_presence)
df_presence = df_presence.fillna(0)
display(df_presence)

,Cluster_0,Cluster_1,Cluster_2,Cluster_3,Cluster_4,Cluster_5,Cluster_6,User
0,17.026194,18.721109,9.013867,5.701079,12.249615,20.647149,16.640986,athenaprocopiou
1,24.406780,13.559322,7.118644,8.474576,7.796610,16.271186,22.372881,daftcollectionofficial
2,21.520803,11.190818,4.447633,26.829268,1.004304,11.621234,23.385940,dodobaror
3,16.750000,16.500000,9.500000,6.000000,13.250000,21.000000,17.000000,emporiosirenuse
4,8.618128,24.368499,18.424963,2.377415,19.019316,17.682021,9.509658,heidikleinswim
5,20.783551,12.670186,4.195610,13.614893,6.446235,17.171437,25.118088,lisamariefernandez
6,14.600410,26.383197,19.979508,3.073770,4.610656,22.950820,8.401639,loupcharmant
7,18.168657,19.109663,10.206298,10.423453,5.573652,21.787912,14.730366,miguelinagambaccini
8,26.816830,9.944751,4.632384,8.499788,5.992350,15.682108,28.431789,muzungusisters
9,11.161116,25.112511,27.992799,4.410441,3.600360,16.381638,11.341134,zeusndione


In [26]:
#Define Cluster Count
cluster_count = 4

#Define Dataset
data_presence = df_presence.copy(deep=True)
del data_presence["User"]

#Implement K-Means Algortihm
model_kmeans = KMeans(n_clusters=cluster_count, random_state=9001)

#Fit Model and Predict
model_kmeans.fit(data_presence)
df_presence['Prediction'] = model_kmeans.labels_
display(df_presence)

,Cluster_0,Cluster_1,Cluster_2,Cluster_3,Cluster_4,Cluster_5,Cluster_6,User,Prediction
0,17.026194,18.721109,9.013867,5.701079,12.249615,20.647149,16.640986,athenaprocopiou,1
1,24.406780,13.559322,7.118644,8.474576,7.796610,16.271186,22.372881,daftcollectionofficial,3
2,21.520803,11.190818,4.447633,26.829268,1.004304,11.621234,23.385940,dodobaror,0
3,16.750000,16.500000,9.500000,6.000000,13.250000,21.000000,17.000000,emporiosirenuse,1
4,8.618128,24.368499,18.424963,2.377415,19.019316,17.682021,9.509658,heidikleinswim,2
5,20.783551,12.670186,4.195610,13.614893,6.446235,17.171437,25.118088,lisamariefernandez,3
6,14.600410,26.383197,19.979508,3.073770,4.610656,22.950820,8.401639,loupcharmant,2
7,18.168657,19.109663,10.206298,10.423453,5.573652,21.787912,14.730366,miguelinagambaccini,1
8,26.816830,9.944751,4.632384,8.499788,5.992350,15.682108,28.431789,muzungusisters,3
9,11.161116,25.112511,27.992799,4.410441,3.600360,16.381638,11.341134,zeusndione,2
